In [23]:
from gloria.datasets.visualization_utils import *
import pandas as pd
import os
import skimage
from torch import nn


def get_attn_overlay(attn, image_shape):
    new_attn = torch.tensor(attn)
#     new_attn = nn.Upsample(size=image_shape)(new_attn.reshape(1, 1, *new_attn.shape))[0, 0]
    new_attn = new_attn.unsqueeze(-1).expand(*new_attn.shape, 3)
    new_attn = skimage.transform.pyramid_expand(
        new_attn, sigma=20, upscale=image_shape[0] // new_attn.shape[0], multichannel=True)
    new_attn = torch.tensor(new_attn[:, :, 0])
    new_attn = nn.Upsample(size=image_shape)(new_attn.reshape(1, 1, *new_attn.shape))[0, 0]
    return new_attn

In [24]:
paths = {
    'original_test_none': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_19_58_28/test_outputs_0/',
    'original_test_swapleftright': '/scratch/mcinerney.de/gloria_outputs4/output/gloria_pretrain_1.0/2021_12_14_13_50_54/test_outputs_0/',
    'original_test_shufflebboxes': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_21_00_23/test_outputs_0/',
    'original_test_randsent': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_20_00_44/test_outputs_0/',
    'original_test_randbboxes': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_20_21_48/test_outputs_0/',
#     'original_none': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_11_05_50/val_outputs_0/',
    'original_none': '/scratch/mcinerney.de/gloria_outputs4/output/gloria_pretrain_1.0/2021_12_07_14_30_56/val_outputs_0/',
    'original_swapleftright': '/scratch/mcinerney.de/gloria_outputs4/output/gloria_pretrain_1.0/2021_12_14_11_59_40/val_outputs_0/',
#     'original_shufflebboxes': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_16_00_31/val_outputs_0/',
    'original_shufflebboxes': '/scratch/mcinerney.de/gloria_outputs4/output/gloria_pretrain_1.0/2021_12_07_16_42_14/val_outputs_0/',
#     'original_randsent': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_10_51_31/val_outputs_0/',
    'original_randsent': '/scratch/mcinerney.de/gloria_outputs4/output/gloria_pretrain_1.0/2021_12_07_14_52_00/val_outputs_0/',
#     'original_randbboxes': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_11_22_47/val_outputs_0/',
    'original_randbboxes': '/scratch/mcinerney.de/gloria_outputs4/output/gloria_pretrain_1.0/2021_12_07_16_14_30/val_outputs_0/',
    'retrained_none': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_19_50_12/val_outputs_0/',
    'retrained_shufflebboxes': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_07_00_24_55/val_outputs_0/',
    'retrained_randsent': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_22_49_28/val_outputs_0/',
    'retrained_randbboxes': '/scratch/mcinerney.de/gloria_outputs3/output/gloria_pretrain_1.0/2021_12_06_23_02_34/val_outputs_0/',
#     'retrained_noattn_loss': '/scratch/mcinerney.de/gloria_outputs/output3/gloria_pretrain_1.0//val_outputs_0/'
}


In [25]:
dfs = {k: pd.read_csv(os.path.join(path, 'sentences.csv')) for k, path in paths.items()}

In [26]:
one_lung_selector = OrSelector(
             RowBBoxSelector(contains={'left lung'}, does_not_contain={'right lung'}),
             RowBBoxSelector(contains={'right lung'}, does_not_contain={'left lung'})
         )
abnormal_selector = RowLabelAndContextSelector(contains={('abnormal', 'yes')})
selectors = {'all': None, 'one_lung': one_lung_selector, 'abnormal': abnormal_selector}
example_df = dfs['original_none']
selector_counts = {name: len(example_df[example_df.apply(selector, axis=1)])
                   if selector is not None else len(example_df)
                   for name, selector in selectors.items()}
example_df = dfs['original_test_none']
selector_counts_test = {name: len(example_df[example_df.apply(selector, axis=1)])
                   if selector is not None else len(example_df)
                   for name, selector in selectors.items()}
score_rows = []
for k, df in dfs.items():
    score_rows.append({'model': k})
    for name, selector in selectors.items():
        rows = df[df.apply(selector, axis=1)] if selector is not None else df
        score_rows[-1]['%s_auroc' % name] = rows.auroc.mean()
        score_rows[-1]['%s_avg_precision' % name] = rows.avg_precision.mean()
        for p in [.1, .2, .3]:
            score_rows[-1]['%s_precision_at_%f' % (name, p)] = rows['precision_at_%f' % p].mean() \
                if 'precision_at_%f' % p in rows.keys() else None
            score_rows[-1]['%s_recall_at_%f' % (name, p)] = rows['recall_at_%f' % p].mean() \
                if 'recall_at_%f' % p in rows.keys() else None
            score_rows[-1]['%s_f1_at_%f' % (name, p)] = rows['f1_at_%f' % p].mean() \
                if 'f1_at_%f' % p in rows.keys() else None
    #     info = path_and_rows_to_info(path, rows=rows)
    #     plot_info(get_attn_overlay, info, path=path)
#     plt.close()
scores_df = pd.DataFrame(score_rows)
print('val counts', selector_counts)
print('test counts', selector_counts_test)
print(scores_df.keys())
scores_df

val counts {'all': 6288, 'one_lung': 1112, 'abnormal': 2611}
test counts {'all': 2496, 'one_lung': 285, 'abnormal': 748}
Index(['model', 'all_auroc', 'all_avg_precision', 'all_precision_at_0.100000',
       'all_recall_at_0.100000', 'all_f1_at_0.100000',
       'all_precision_at_0.200000', 'all_recall_at_0.200000',
       'all_f1_at_0.200000', 'all_precision_at_0.300000',
       'all_recall_at_0.300000', 'all_f1_at_0.300000', 'one_lung_auroc',
       'one_lung_avg_precision', 'one_lung_precision_at_0.100000',
       'one_lung_recall_at_0.100000', 'one_lung_f1_at_0.100000',
       'one_lung_precision_at_0.200000', 'one_lung_recall_at_0.200000',
       'one_lung_f1_at_0.200000', 'one_lung_precision_at_0.300000',
       'one_lung_recall_at_0.300000', 'one_lung_f1_at_0.300000',
       'abnormal_auroc', 'abnormal_avg_precision',
       'abnormal_precision_at_0.100000', 'abnormal_recall_at_0.100000',
       'abnormal_f1_at_0.100000', 'abnormal_precision_at_0.200000',
       'abnormal_recall_

,model,all_auroc,all_avg_precision,all_precision_at_0.100000,all_recall_at_0.100000,all_f1_at_0.100000,all_precision_at_0.200000,all_recall_at_0.200000,all_f1_at_0.200000,all_precision_at_0.300000,...,abnormal_avg_precision,abnormal_precision_at_0.100000,abnormal_recall_at_0.100000,abnormal_f1_at_0.100000,abnormal_precision_at_0.200000,abnormal_recall_at_0.200000,abnormal_f1_at_0.200000,abnormal_precision_at_0.300000,abnormal_recall_at_0.300000,abnormal_f1_at_0.300000
0,original_test_none,0.690680,0.516796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.482862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,original_test_swapleftright,0.690090,0.516276,0.590486,0.394454,0.411409,0.596240,0.508007,0.499862,0.548927,...,0.480868,0.539606,0.407875,0.406541,0.545306,0.532213,0.496276,0.514405,0.630784,0.507914
2,original_test_shufflebboxes,0.687235,0.516795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.477410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,original_test_randsent,0.681315,0.509981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.467818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,original_test_randbboxes,0.631343,0.452842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.411582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,original_none,0.643243,0.437247,0.489153,0.370550,0.372345,0.499430,0.481672,0.450770,0.464862,...,0.423967,0.469585,0.367089,0.362343,0.475996,0.485347,0.442914,0.445772,0.593307,0.460164
6,original_swapleftright,0.642606,0.436547,0.489442,0.368092,0.371742,0.499881,0.479107,0.450292,0.464355,...,0.422460,0.468989,0.359935,0.359340,0.477413,0.480475,0.442129,0.444978,0.596939,0.458457
7,original_shufflebboxes,0.639618,0.436297,0.488960,0.365633,0.368410,0.496244,0.474160,0.445851,0.463011,...,0.419648,0.470332,0.356831,0.358282,0.475839,0.467293,0.435134,0.437220,0.597945,0.451166
8,original_randsent,0.633379,0.430579,0.488724,0.336421,0.352056,0.496707,0.453659,0.436145,0.450137,...,0.410879,0.469590,0.329213,0.342823,0.475241,0.443370,0.422521,0.420889,0.613106,0.442769
9,original_randbboxes,0.608374,0.417647,0.462442,0.332895,0.344657,0.468499,0.434411,0.419027,0.442589,...,0.398219,0.439363,0.325661,0.332540,0.445474,0.425447,0.402842,0.420242,0.550559,0.431695


In [27]:
latex_table = ''
model_names = {'original_test': 'GLoRIA', 'original': 'GLoRIA', 'retrained': 'GLoRIA retrained'}
augmentation_names = {'none': 'None', 'randsent': 'Random Sentences',
                      'shufflebboxes': 'Shuffle Bboxes', 'randbboxes': 'Random Bboxes',
                      'swapleftright': 'Swap Left Right'}
for i, row in scores_df.iterrows():
    formatted_row_scores = []
    for name in ['all']:
        formatted_row_scores.append(row['%s_auroc' % name])
        formatted_row_scores.append(row['%s_avg_precision' % name])
        formatted_row_scores.append('%.2f/%.2f/%.2f' % tuple(row['%s_precision_at_%f' % (name, p)] * 100 for p in [.1, .2, .3]))
#             formatted_row_scores.append(row['%s_recall_at_%f' % (name, p)])
#             formatted_row_scores.append(row['%s_f1_at_%f' % (name, p)])
    augmentation = augmentation_names[row.model.split('_')[-1]]
    model = model_names['_'.join(row.model.split('_')[:-1])]
    formatted_row_scores = '%s & %s & ' % (model, augmentation) + ' & '.join(['%.2f' % (s * 100) if isinstance(s, float) else str(s) for s in formatted_row_scores])
    latex_table += formatted_row_scores + ' \\\\\n'
print(latex_table)

GLoRIA & None & 69.07 & 51.68 & nan/nan/nan \\
GLoRIA & Swap Left Right & 69.01 & 51.63 & 59.05/59.62/54.89 \\
GLoRIA & Shuffle Bboxes & 68.72 & 51.68 & nan/nan/nan \\
GLoRIA & Random Sentences & 68.13 & 51.00 & nan/nan/nan \\
GLoRIA & Random Bboxes & 63.13 & 45.28 & nan/nan/nan \\
GLoRIA & None & 64.32 & 43.72 & 48.92/49.94/46.49 \\
GLoRIA & Swap Left Right & 64.26 & 43.65 & 48.94/49.99/46.44 \\
GLoRIA & Shuffle Bboxes & 63.96 & 43.63 & 48.90/49.62/46.30 \\
GLoRIA & Random Sentences & 63.34 & 43.06 & 48.87/49.67/45.01 \\
GLoRIA & Random Bboxes & 60.84 & 41.76 & 46.24/46.85/44.26 \\
GLoRIA retrained & None & 57.04 & 39.61 & nan/nan/nan \\
GLoRIA retrained & Shuffle Bboxes & 56.28 & 38.88 & nan/nan/nan \\
GLoRIA retrained & Random Sentences & 55.25 & 37.83 & nan/nan/nan \\
GLoRIA retrained & Random Bboxes & 55.31 & 38.67 & nan/nan/nan \\



In [28]:
latex_table = ''
model_names = {'original_test': 'GLoRIA', 'original': 'GLoRIA', 'retrained': 'GLoRIA retrained'}
augmentation_names = {'none': 'None', 'randsent': 'Random Sentences',
                      'shufflebboxes': 'Shuffle Bboxes', 'randbboxes': 'Random Bboxes',
                      'swapleftright': 'Swap Left Right'}
for i, row in scores_df.iterrows():
    formatted_row_scores = []
    for name in ['one_lung']:
        formatted_row_scores.append(row['%s_auroc' % name])
        formatted_row_scores.append(row['%s_avg_precision' % name])
        formatted_row_scores.append('%.2f/%.2f/%.2f' % tuple(row['%s_precision_at_%f' % (name, p)] * 100 for p in [.1, .2, .3]))
#             formatted_row_scores.append(row['%s_recall_at_%f' % (name, p)])
#             formatted_row_scores.append(row['%s_f1_at_%f' % (name, p)])
    augmentation = augmentation_names[row.model.split('_')[-1]]
    model = model_names['_'.join(row.model.split('_')[:-1])]
    formatted_row_scores = '%s & %s & ' % (model, augmentation) + ' & '.join(['%.2f' % (s * 100) if isinstance(s, float) else str(s) for s in formatted_row_scores])
    latex_table += formatted_row_scores + ' \\\\\n'
print(latex_table)

GLoRIA & None & 65.48 & 38.67 & nan/nan/nan \\
GLoRIA & Swap Left Right & 65.24 & 38.40 & 42.48/43.42/39.33 \\
GLoRIA & Shuffle Bboxes & 65.41 & 39.01 & nan/nan/nan \\
GLoRIA & Random Sentences & 64.97 & 37.88 & nan/nan/nan \\
GLoRIA & Random Bboxes & 59.94 & 35.12 & nan/nan/nan \\
GLoRIA & None & 61.75 & 31.91 & 34.59/35.10/33.00 \\
GLoRIA & Swap Left Right & 61.53 & 31.57 & 34.52/35.04/32.64 \\
GLoRIA & Shuffle Bboxes & 61.40 & 31.61 & 34.33/34.25/32.06 \\
GLoRIA & Random Sentences & 60.55 & 30.90 & 33.85/34.36/31.61 \\
GLoRIA & Random Bboxes & 57.44 & 26.95 & 28.48/29.07/27.44 \\
GLoRIA retrained & None & 56.23 & 29.27 & nan/nan/nan \\
GLoRIA retrained & Shuffle Bboxes & 54.80 & 27.74 & nan/nan/nan \\
GLoRIA retrained & Random Sentences & 53.66 & 26.52 & nan/nan/nan \\
GLoRIA retrained & Random Bboxes & 53.96 & 25.78 & nan/nan/nan \\



In [29]:
latex_table = ''
model_names = {'original_test': 'GLoRIA', 'original': 'GLoRIA', 'retrained': 'GLoRIA retrained'}
augmentation_names = {'none': 'None', 'randsent': 'Random Sentences',
                      'shufflebboxes': 'Shuffle Bboxes', 'randbboxes': 'Random Bboxes',
                      'swapleftright': 'Swap Left Right'}
for i, row in scores_df.iterrows():
    formatted_row_scores = []
    for name in ['abnormal']:
        formatted_row_scores.append(row['%s_auroc' % name])
        formatted_row_scores.append(row['%s_avg_precision' % name])
        formatted_row_scores.append('%.2f/%.2f/%.2f' % tuple(row['%s_precision_at_%f' % (name, p)] * 100 for p in [.1, .2, .3]))
#             formatted_row_scores.append(row['%s_recall_at_%f' % (name, p)])
#             formatted_row_scores.append(row['%s_f1_at_%f' % (name, p)])
    augmentation = augmentation_names[row.model.split('_')[-1]]
    model = model_names['_'.join(row.model.split('_')[:-1])]
    formatted_row_scores = '%s & %s & ' % (model, augmentation) + ' & '.join(['%.2f' % (s * 100) if isinstance(s, float) else str(s) for s in formatted_row_scores])
    latex_table += formatted_row_scores + ' \\\\\n'
print(latex_table)

GLoRIA & None & 69.51 & 48.29 & nan/nan/nan \\
GLoRIA & Swap Left Right & 69.34 & 48.09 & 53.96/54.53/51.44 \\
GLoRIA & Shuffle Bboxes & 68.65 & 47.74 & nan/nan/nan \\
GLoRIA & Random Sentences & 67.42 & 46.78 & nan/nan/nan \\
GLoRIA & Random Bboxes & 61.95 & 41.16 & nan/nan/nan \\
GLoRIA & None & 64.40 & 42.40 & 46.96/47.60/44.58 \\
GLoRIA & Swap Left Right & 64.30 & 42.25 & 46.90/47.74/44.50 \\
GLoRIA & Shuffle Bboxes & 63.76 & 41.96 & 47.03/47.58/43.72 \\
GLoRIA & Random Sentences & 62.78 & 41.09 & 46.96/47.52/42.09 \\
GLoRIA & Random Bboxes & 60.19 & 39.82 & 43.94/44.55/42.02 \\
GLoRIA retrained & None & 56.61 & 37.65 & nan/nan/nan \\
GLoRIA retrained & Shuffle Bboxes & 55.61 & 36.50 & nan/nan/nan \\
GLoRIA retrained & Random Sentences & 54.34 & 35.11 & nan/nan/nan \\
GLoRIA retrained & Random Bboxes & 54.81 & 36.75 & nan/nan/nan \\



In [ ]:
matching_scores = []
for base in ['original', 'original_test', 'retrained']:
    matching_scores.append({'model': base})
    combined = pd.merge(dfs['%s_none' % base], dfs['%s_randsent' % base].rename(
        columns=lambda x: 'neg_' + x), left_on='dicom_sent_id', right_on='neg_dicom_sent_id')
    for n, selector in selectors.items():
        rows = combined[combined.apply(selector, axis=1)] if selector is not None else combined
        matching_scores[-1]['%s_local_acc' % n] = (rows.local_sims > rows.neg_local_sims).mean()
        matching_scores[-1]['%s_global_acc' % n] = (rows.global_sims > rows.neg_global_sims).mean()
matching_scores = pd.DataFrame(matching_scores)
latex_table = ''
model_names = {'original_test': 'GLoRIA', 'original': 'GLoRIA', 'retrained': 'GLoRIA retrained'}
for i, row in matching_scores.iterrows():
    latex_table += '%s & ' % model_names[row.model] + ' & '.join([
        '%.1f' % (row[col] * 100) for col in [
            'all_local_acc', 'all_global_acc',
            'one_lung_local_acc', 'one_lung_global_acc',
            'abnormal_local_acc', 'abnormal_global_acc',
        ]
    ]) + ' \\\\\n'
print(latex_table)
matching_scores

In [ ]:
correlation_rows = []
for base in ['original', 'original_test', 'retrained']:
    correlation_rows.append({'model': base})
    local_bce = np.log((pd.concat([dfs['%s_none' % base].local_sims / 5, 1 - (dfs['%s_randsent' % base].local_sims / 5)]) + 1) / 2)
    global_bce = np.log((pd.concat([dfs['%s_none' % base].global_sims, 1 - dfs['%s_randsent' % base].global_sims]) + 1) / 2)
#     local_bce = (pd.concat([dfs['%s_none' % base].local_sims / 5, 1 - (dfs['%s_randsent' % base].local_sims / 5)]) + 1) / 2
#     global_bce = (pd.concat([dfs['%s_none' % base].global_sims, 1 - dfs['%s_randsent' % base].global_sims]) + 1) / 2
    avg_precision = pd.concat([dfs['%s_none' % base].avg_precision, dfs['%s_randsent' % base].avg_precision])
    auroc = pd.concat([dfs['%s_none' % base].auroc, dfs['%s_randsent' % base].auroc])
    attn_entropy = pd.concat([dfs['%s_none' % base].attn_entropy, dfs['%s_randsent' % base].attn_entropy])
    correlation_rows[-1]['local_matching_bce_and_global_matching_bce'] = local_bce.corr(global_bce)
    for name, bce in [('local', local_bce), ('global', global_bce)]:
        correlation_rows[-1]['%s_matching_bce_and_attn_entropy' % name] = bce.corr(attn_entropy)
        correlation_rows[-1]['%s_matching_bce_and_avg_precision' % name] = bce.corr(avg_precision)
        correlation_rows[-1]['%s_matching_bce_and_auroc' % name] = bce.corr(auroc)
    correlation_rows[-1]['attn_entropy_and_auroc'] = attn_entropy.corr(auroc)
    correlation_rows[-1]['attn_entropy_and_avg_precision'] = attn_entropy.corr(avg_precision)
    correlation_rows[-1]['auroc_and_avg_precision'] = auroc.corr(avg_precision)
correlation_df = pd.DataFrame(correlation_rows)
latex_table = ''
model_names = {'original_test': 'GLoRIA', 'original': 'GLoRIA', 'retrained': 'GLoRIA retrained'}
metrics = ['local_matching_bce', 'global_matching_bce', 'attn_entropy', 'auroc', 'avg_precision']
metric_names = {
    'local_matching_bce': 'Local Matching BCE',
    'global_matching_bce': 'Global Matching BCE',
    'attn_entropy': 'Attn Entropy',
    'auroc': 'AUROC',
    'avg_precision': 'Avg Precision',
}
latex_table = '& ' + ' & '.join([metric_names[m] for m in metrics[:-1]]) + ' \\\\\n'
for i, row in correlation_df.iterrows():
    latex_table += '\\hline\n%s \\\\\n\\hline\n' % model_names[row.model]
    for metric2 in metrics[1:]:
        latex_table += '%s & ' % metric_names[metric2] + ' & '.join([
            '%.3f' % row[(metric1 + '_and_' + metric2)]
            if (metric1 + '_and_' + metric2) in row.keys() else ''
            for metric1 in metrics[:-1]
        ]) + ' \\\\\n'
print(latex_table)
correlation_df


In [ ]:
import os
from jupyter_innotater import *
import pandas as pd


def visualize(model_paths, selector=None):
    min_list_of_files = None
    for model_path in model_paths.values():
        fs = set(os.listdir(os.path.join(model_path, 'sentence_figures')))
        df = pd.read_csv(os.path.join(model_path, 'sentences.csv'))
        if selector is not None:
            df = df[df.apply(selector, axis=1)]
        fs = fs.intersection(set([dicom_sent_id + '.jpg' for dicom_sent_id in set(df.dicom_sent_id)]))
        if min_list_of_files is None:
            min_list_of_files = fs
        else:
            min_list_of_files = min_list_of_files.intersection(fs)
    min_list_of_files = sorted(list(min_list_of_files))
    innotations = [
        innotation
        for model_name, model_path in model_paths.items()
        for innotation in [
            TextInnotation([model_name] * len(min_list_of_files)),
            TextInnotation(min_list_of_files),
            ImageInnotation(min_list_of_files, path=os.path.join(model_path, 'sentence_figures'))
        ]
    ]
    return Innotater(
        innotations,
        []
    )

In [ ]:
visualize(paths, selector=selector)

In [ ]:
from omegaconf import OmegaConf
import gloria
from gloria.datasets.mimic_for_gloria import GloriaCollateFn
from gloria.lightning.callbacks import EvaluateLocalization
from gloria.lightning.pretrain_model import PretrainModel
import matplotlib.pyplot as plt

def get_instance(dicom_id, sent_id, dataset, replace_sent_with=None):
    rows = dataset.df[dataset.df.dicom_id == dicom_id]
    instance = dataset.get_item_from_rows(rows)
    instance = dataset.add_objects(instance, sent_id=sent_id)
    if replace_sent_with is not None:
        patient_id = next(iter(instance.keys()))
        study_id = next(iter(instance[patient_id].keys()))
        instance[patient_id][study_id]['sentence'] = replace_sent_with
        instance[patient_id][study_id]['objects'][dicom_id]['sent_to_bboxes'][sent_id]['sentence'] = replace_sent_with
    return instance

def display(instance):
    patient_id = next(iter(instance.keys()))
    study_id = next(iter(instance[patient_id].keys()))
    print(instance[patient_id][study_id]['sentence'])
    img = instance[patient_id][study_id]['images'][dicom_id]
    plt.imshow(to_rgb(img))
    

In [ ]:
cfg = OmegaConf.load('configs/imagenome_pretrain_val_config.yaml')
dm = gloria.builder.build_data_module(cfg)
collate_fn = GloriaCollateFn(cfg, "test")
el = EvaluateLocalization(collate_fn)
module = PretrainModel.load_from_checkpoint('./pretrained/chexpert_resnet50.ckpt')
module.eval()
valid = dm.dm.get_dataset('valid')

In [ ]:
dicom_id = '2533ab2a-565051cb-35201672-0267a457-d931d20f'
sent_id = '55609974|6'
instance = get_instance(dicom_id, sent_id, valid)
display(instance)
el.evaluate_and_save(
    path='experiments2', instances=[instance], pl_module=module,
    save_full_data=True, plot=True, plot_attn_overlay_mode='pyramid')

In [ ]:
neg_instance = get_instance(dicom_id, sent_id, valid, replace_sent_with='Increased consolidation in the right lung.')
display(neg_instance)
el.evaluate_and_save(
    path='experiments4', instances=[neg_instance], pl_module=module,
    save_full_data=True, plot=True, plot_attn_overlay_mode='pyramid')

In [ ]:
visualize({'regular': 'experiments', 'reversed': 'experiments_neg'})

In [ ]:
neg_instance = get_instance(dicom_id, sent_id, valid, replace_sent_with='There left lung is normal.')
display(neg_instance)
el.evaluate_and_save(
    path='experiments4', instances=[neg_instance], pl_module=module,
    save_full_data=True, plot=True, plot_attn_overlay_mode='pyramid')

In [ ]:
valid[0]

In [ ]:
gold = dm.dm.get_dataset('gold')

In [ ]:
class GenerateContextLocationConditionSentences:
    def __call__(self, contexts, locations, conditions):
        condition_to_locations = {}
        for context, loc, condition in zip(contexts, locations, conditions):
            if condition not in condition_to_locations.keys():
                condition_to_locations[condition] = []
            if context == 'yes':
                condition_to_locations[condition].append(loc)
        sentence = ''
        for cond, locs in condition_to_locations.items():
            new_locs = set()
            for loc in locs:
                if ('left' in loc or 'right' in loc) and (loc.replace('left', 'right') in locs or loc.replace('right', 'left') in locs):
                    new_locs.add(loc.replace('left ', '').replace('right ', '') + 's')
                else:
                    new_locs.add(loc)
            new_locs = list(new_locs)
            if len(locs) == 0:
                sentence += ' There is no ' + cond + '.'
            else:
                if len(new_locs) > 2:
                    loclist = ', '.join(new_locs[:-1]) + ', and ' + new_locs[-1]
                else:
                    loclist = ' and '.join(new_locs)
                if cond == 'normal' or cond == 'abnormal':
                    sentence += ' The ' + loclist + (' are ' if len(locs) > 1 else ' is ') + cond + '.'
                else:
                    sentence += ' There is ' + cond + ' in the ' + loclist + '.'
        return sentence.strip()

In [ ]:
generate_sentence = GenerateContextLocationConditionSentences()
for i in tqdm(range(len(gold)), total=len(gold)):
    instance = gold[i]
    patient_id = next(iter(instance.keys()))
    study_id = next(iter(instance[patient_id].keys()))
    dicom_id = next(iter(instance[patient_id][study_id]['images'].keys()))
    for k, v in instance[patient_id][study_id]['objects'][dicom_id]['sent_to_bboxes'].items():
        print(v['sentence'])
        print(generate_sentence(v['contexts'], v['bboxes'], v['labels']))
        print()

In [ ]:
set([c for c, _ in valid_label_bbox_pairs])

In [ ]:
set([c for c, _ in valid_label_bbox_pairs])